#### Copyright 2018 Google LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Natural Language Understanding: WordNet

Please **make a copy** of this Colab notebook before starting this lab. To do so, choose **File**->**Save a copy in Drive**.

## Topics covered
  1. Synsets
  1. Lemmas and synonyms
  1. Word hierarchies
  1. Measuring similarities

One of the earliest attempts to create useful representations of meaning for language is [WordNet](https://en.wikipedia.org/wiki/WordNet) -- a lexical database of words and their relationships.

NLTK provides a [WordNet wrapper](http://www.nltk.org/howto/wordnet.html) that we'll use here.

In [ ]:
import nltk
assert(nltk.download('wordnet'))  # Make sure we have the wordnet data.
from nltk.corpus import wordnet as wn

## Synsets
The fundamental WordNet unit is a **synset**, specified by a word form, a part of speech, and an index. The synsets() function retrieves the synsets that match the given word. For example, there are 4 synsets for the word "surf", one of which is a noun (n) and three of which are verbs (v). WordNet provides a definition and sometimes glosses (examples) for each synset. **Polysemy**, by the way, means having multiple senses.

In [ ]:
for s in wn.synsets('surf'):
    print s
    print '\t', s.definition()
    print '\t', s.examples()

## Lemmas and synonyms
Each synset includes its corresponding **lemmas** (word forms).

We can construct a set of synonyms by looking up all the lemmas for all the synsets for a word.

In [ ]:
synonyms = set()

for s in wn.synsets('triumphant'):
    for l in s.lemmas():
        synonyms.add(l.name())

print 'synonyms:', ', '.join(synonyms)

## Word hierarchies
WordNet organizes nouns and verbs into hierarchies according to **hypernym** or is-a relationships.

Let's examine the path from "rutabaga" to its root in the tree, "entity".

In [ ]:
s = wn.synsets('rutabaga')

while s:
    print s[0].hypernyms()
    s = s[0].hypernyms()

Actually, the proper way to do this is with a transitive closure, which repeatedly applies the specified function (in this case, hypernyms()).

In [ ]:
hyper = lambda x: x.hypernyms()
s = wn.synset('rutabaga.n.01')
for i in list(s.closure(hyper)):
    print i
print
ss = wn.synset('root_vegetable.n.01')
for i in list(ss.closure(hyper)):
    print i

## Measuring similarity

WordNet's word hierarchies (for nouns and verbs) allow us to measure similarity in various ways.

Path similarity is defined as:

> $1 / (ShortestPathDistance(s_1, s_2) + 1)$

where $ShortestPathDistance(s_1, s_2)$ is computed from the hypernym/hyponym graph.



In [ ]:
s1 = wn.synset('dog.n.01')
s2 = wn.synset('cat.n.01')
s3 = wn.synset('potato.n.01')

print s1, '::', s1, s1.path_similarity(s1)
print s1, '::', s2, s1.path_similarity(s2)
print s1, '::', s3, s1.path_similarity(s3)
print s2, '::', s3, s2.path_similarity(s3)

print

hyper = lambda x: x.hypernyms()

print(s1.hypernyms())

for i in list(s1.closure(hyper)):
    print i

## Takeaways

WordNet gives us ways to compare words and understand their relationships in a much more meaningful way than relying on the raw strings (sequences of characters). We know that 'cat' and 'dog', for example, are somewhat similar even though they have no string similarity. As a result, WordNet has been used in lots of practical applications over the years. However, WordNet has a few important shortcomings:

1. WordNet was built by people. This makes it hard to maintain as new words are added (e.g. 'iphone' isn't in WordNet) and definitions evolve. It also has limited language coverage. NLTK wraps Open Multilingual WordNet which includes 22 additional languages, but these are less extensive than the English WordNet. A fundamental question addressed by subsequent sections is: can we build WordNet-like resources automatically from text, of which there is an abundance?

1. WordNet, like any dictionary or thesaurus, represents the meaning of a word with its relationships to other words. That is, it lacks *grounding* in the real world. This is fine for people who have plenty of working knowledge of the world, who have seen and interacted with dogs and cats and potatoes, but would be much less helpful for aliens arriving on Earth for the first time. This deficiency, where language is only defined with respect to itself, and not with respect to images for example, is at the frontier of research in Natural Language Understanding.

## Quiz Questions

(1) Use the closure function to enumerate the **hyponyms** (the inverse of a hypernym) of 'root_vegetable.n.01'.

(2) We used the path_similarity function to compute the similarity between 'dog' and 'cat'. Use the hypernyms() function (see above) to find the path between these two words. Does the path similarity 0.2 make sense?